In [1]:
import os
from dotenv import load_dotenv, find_dotenv

from datetime import datetime
from collections import OrderedDict

import sys
from pathlib import Path, PurePath as PPath

print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        # NB: Path does not have a str method.
        newp = str(PPath(this_path).parent)
    else:
        newp = str(PPath(this_path)) 
    
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()

DATA_RAW = DIR_DATA.joinpath('raw')
DATA_INTERIM = DIR_DATA.joinpath('intermediate')
DATA_READY = DIR_DATA.joinpath('production')

def data_subfolders():
    ans = int(input('Setup data subfolders (raw, intermediate, production)?  1=yes, 0=no: '))
    if ans == 1:
        for d in [DATA_RAW, DATA_INTERIM, DATA_READY]:
            if not d.exists():
                Path.mkdir(d)
                print(f'Created: {d}')
                
import numpy as np
import scipy as sp
from scipy import stats as sps
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", 300)

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-muted')

from pprint import pprint as pp

# Filtered dir() for method discovery:
def filter_dir(obj, start_with_str='_', exclude=True):
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]

def get_mdl_pkgs(alib):
    import inspect
    "Inspect module hierarchy on two levels ony."
    for name, mdl in inspect.getmembers(alib, inspect.ismodule):
        print('\n{:>13} : {}'.format(mdl.__name__, filter_dir(mdl)))
        for mdl_name, mdl_sub in inspect.getmembers(mdl, inspect.ismodule):
            if mdl_sub.__doc__:
                print('\n{:>20} : {}'.format(mdl_name, mdl_sub.__doc__.strip()))
                

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import HTML, Markdown #, IFrame
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))


Markdown( """**First code cell from custom template `dsml.ipynb`**. 
_See Tim Paine's [jupyter_lab templates extention](https://github.com/timkpaine/jupyterlab_templates)._<br>
""")

def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    return HTML('<div style="{}">{}</div>'.format(style, title))


def add_div(div_class, div_start, div_text, output_string=True):
    from IPython import get_ipython
    from IPython.display import HTML, Markdown
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output string, but the cell mode is still in 'code' not 'markdown':
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    The only thing to do is change the cell mode to Markdown.
    If `output_string=False`, the HTML output is displayed in an output cell.
    """
    accepted = ['alert-info', 'alert-warning', 'alert-danger']
    if div_class not in accepted:
        return HTML(f"""<div class="alert"><b>Wrong class:</b> `div_start` is one of {accepted}.
                    </div>""")
    div = f"""<div class="alert {div_class}"><b>{div_start}</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div) #HTML(div)

# autoreload extension
from IPython import get_ipython
ipython = get_ipython()

if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

Python ver: 3.6.7 (default, Feb 28 2019, 07:28:18) [MSC v.1900 64 bit (AMD64)]
Python env: dsml
Currrent dir: C:\Users\catch\Documents\GH_Projects\NYC_data

Path added to sys.path: C:\Users\catch\Documents\GH_Projects\NYC_data


In [ ]:
data_subfolders()

---
---
# NYC OpenData - Datasets
---

In [2]:
import sodapy
SODA_VER = '2.1'

# local utils lib
import etl

In [3]:
# load variables from APIs user account(s):
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

if dotenv_path == '':
    print("""Local '.env' file not found:\n\tThe Socrata credential variables are set to None, 
    which may not be accepted.\n\tIf that's the case, create the file with the credentials as KEY=value on each line.""")
else:
    # load up the entries as environment variables
    load_dotenv(dotenv_path)
    
    #GOOGLE_KEY = os.getenv("GOO_GEO_API_1")
    SODA_U = os.environ.get('SODA_U')
    SODA_P = os.environ.get('SODA_P')
    SODA_SECRET = os.environ.get('SODA_SECRET')
    SODA_APP_TOKEN=os.environ.get('SODA_APP_TOKEN')

True

In [4]:
SODA_NYC = 'data.cityofnewyork.us'
cli_NYC = sodapy.Socrata(SODA_NYC, SODA_APP_TOKEN, username=SODA_U, password=SODA_P)

---
# json hook examples

In [25]:
# %load "complex_data.json"
[
  {
    "__complex__":true,
    "real":42,
    "imag":36
  },
  {
    "__complex__":true,
    "real":64,
    "imag":11
  }
]

In [16]:
def decode_complex(dct):
    if "__complex__" in dct:
        return complex(dct["real"], dct["imag"])
    return dct

In [19]:
with open("complex_data.json") as complex_data:
    data = complex_data.read()
    z = json.loads(data, object_hook=decode_complex)

type(z)
z

list

[(42+36j), (64+11j)]

---
# example (encoder) from astropy:

In [ ]:
class DomainDatasets():
    """
    Class to obtain the datasets listed by a domain in sodapy.socrata.client.
    Includes a decoding function to deserialize the json output for 
    proper loading into pandas.DataFrame.
    """
    def __init__(self, cli):
        self.socrata_cli = cli
        self.decoder = json_decode_hook
        
        
    def get_datasets(self):
        sets = self.socrata_cli.datasets()
        
        return json.JSONDecoder
        
        
    def json_decode_hook():
        

In [12]:
import json
import pandas.io.json as pdjson

In [9]:
raw_datasets = DATA_RAW.joinpath('NYC_datasets.json')

NYC_datasets = etl.get_domain_datasets(cli_NYC, local_file=raw_datasets, replace=False)
len(NYC_datasets)

2822

In [14]:
nyc_datasets = pdjson.read_json(raw_datasets)
nyc_datasets.shape

nyc_datasets.head()

(2823, 7)

,classification,link,metadata,owner,permalink,preview_image_url,resource
0,"{'categories': ['economy', 'environment', 'housing & development'], 'tags': [], 'domain_category': 'Housing & Development', 'domain_tags': ['job', 'dob', 'buildings'], 'domain_metadata': [{'key': 'Update_Automation', 'value': 'Yes'}, {'key': 'Update_Date-Made-Public', 'value': '4/26/2013'}, {'ke...",https://data.cityofnewyork.us/Housing-Development/DOB-Job-Application-Filings/ic3t-wcy2,{'domain': 'data.cityofnewyork.us'},"{'id': '5fuc-pqz2', 'display_name': 'NYC OpenData'}",https://data.cityofnewyork.us/d/ic3t-wcy2,NaN,"{'name': 'DOB Job Application Filings', 'id': 'ic3t-wcy2', 'parent_fxf': None, 'description': 'This dataset contains all job applications submitted through the Borough Offices, through eFiling, or through the HUB, which have a ""Latest Action Date"" since January 1, 2000. This dataset does not inc..."
1,"{'categories': ['transportation', 'environment'], 'tags': [], 'domain_category': 'Transportation', 'domain_tags': ['new application', 'fhv', 'medallion', 'green', 'shl', 'street hail livery', 'for-hire', 'exam', 'results', 'limousine', 'status', 'driver', 'exam fee', 'fingerprint', 'hack number'...",https://data.cityofnewyork.us/Transportation/TLC-New-Driver-Application-Status/dpec-ucu7,{'domain': 'data.cityofnewyork.us'},"{'id': '5fuc-pqz2', 'display_name': 'NYC OpenData'}",https://data.cityofnewyork.us/d/dpec-ucu7,NaN,"{'name': 'TLC New Driver Application Status', 'id': 'dpec-ucu7', 'parent_fxf': None, 'description': 'THIS DATASET IS UPDATED SEVERAL TIMES PER DAY. TLC Driver application status check for applicants who had applied for a new TLC driver’s license. For more information and to upload missing requir..."
2,"{'categories': [], 'tags': [], 'domain_category': 'City Government', 'domain_tags': ['2018od4a-report', '2018od4a-video'], 'domain_metadata': [{'key': 'Update_Automation', 'value': 'Yes'}, {'key': 'Update_Date-Made-Public', 'value': '7/12/2016'}, {'key': 'Update_Update-Frequency', 'value': 'Dail...",https://data.cityofnewyork.us/City-Government/Civil-Service-List-Active-/vx8i-nprf,{'domain': 'data.cityofnewyork.us'},"{'id': '5fuc-pqz2', 'display_name': 'NYC OpenData'}",https://data.cityofnewyork.us/d/vx8i-nprf,NaN,"{'name': 'Civil Service List (Active)', 'id': 'vx8i-nprf', 'parent_fxf': None, 'description': 'A Civil Service List consists of all candidates who passed an exam, ranked in score order. An established list is considered active for no less than one year and no more than four years from the date o..."
3,"{'categories': [], 'tags': [], 'domain_category': 'Transportation', 'domain_tags': ['fhv', 'for hire', 'for-hire', 'for-hire-vehicles', 'drivers', 'taxi', 'active', 'inactive'], 'domain_metadata': [{'key': 'Update_Automation', 'value': 'Yes'}, {'key': 'Update_Date-Made-Public', 'value': '7/20/20...",https://data.cityofnewyork.us/Transportation/For-Hire-Vehicles-FHV-Active/8wbx-tsch,{'domain': 'data.cityofnewyork.us'},"{'id': '5fuc-pqz2', 'display_name': 'NYC OpenData'}",https://data.cityofnewyork.us/d/8wbx-tsch,NaN,"{'name': 'For Hire Vehicles (FHV) - Active', 'id': '8wbx-tsch', 'parent_fxf': None, 'description': 'TLC authorized For-Hire vehicles that are active. This list is accurate to the date and time represented in the Last Date Updated and Last Time Updated fields. For inquiries about the contents of ..."
4,"{'categories': [], 'tags': [], 'domain_category': 'Transportation', 'domain_tags': ['fhv', 'for hire', 'for-hire', 'for-hire-vehicles', 'drivers', 'taxi', 'active'], 'domain_metadata': [{'key': 'Update_Automation', 'value': 'Yes'}, {'key': 'Update_Date-Made-Public', 'value': '7/20/2015'}, {'key'...",https://data.cityofnewyork.us/Transportation/For-Hire-Vehicles-FHV-Active-Drivers/xjfq-wh2d,{'domain': 'data.cityofnewyork.us'},"{'id': '5fuc-pqz2', 'display_name': 'NYC OpenData'}",https://data.cityofnewyork.us/d/xjfq-wh2d,NaN,"{'name': 'For Hire Vehicles (FHV) - Active Drivers', 'id': 'xjfq-wh2d', 'parent_

## Need to split subfields

In [28]:
json_norm = pdjson.json_normalize

In [11]:
NYC_datasets[0].keys()

dict_keys(['resource', 'classification', 'metadata', 'permalink', 'link', 'owner'])

In [34]:
for k, v in NYC_datasets[0].items():
    if isinstance(v, dict):
        print(f'{k}:\n\tnext keys:\n\t{v.keys()}')
    else:
        print(f'{k}')

resource:
	next keys:
	dict_keys(['name', 'id', 'parent_fxf', 'description', 'attribution', 'attribution_link', 'contact_email', 'type', 'updatedAt', 'createdAt', 'metadata_updated_at', 'data_updated_at', 'page_views', 'columns_name', 'columns_field_name', 'columns_datatype', 'columns_description', 'columns_format', 'download_count', 'provenance', 'lens_view_type', 'blob_mime_type', 'hide_from_data_json', 'publication_date'])
classification:
	next keys:
	dict_keys(['categories', 'tags', 'domain_category', 'domain_tags', 'domain_metadata'])
metadata:
	next keys:
	dict_keys(['domain'])
permalink
link
owner:
	next keys:
	dict_keys(['id', 'display_name'])


In [54]:
df = json_norm(NYC_datasets, sep="_")
df.head()

,classification_categories,classification_domain_category,classification_domain_metadata,classification_domain_private_metadata,classification_domain_tags,classification_tags,link,metadata_additional_access_points,metadata_domain,metadata_license,owner_display_name,owner_id,permalink,preview_image_url,resource_attribution,resource_attribution_link,resource_blob_mime_type,resource_columns_datatype,resource_columns_description,resource_columns_field_name,resource_columns_format,resource_columns_name,resource_contact_email,resource_createdAt,resource_data_updated_at,resource_description,resource_download_count,resource_hide_from_data_json,resource_id,resource_lens_view_type,resource_metadata_updated_at,resource_name,resource_page_views_page_views_last_month,resource_page_views_page_views_last_month_log,resource_page_views_page_views_last_week,resource_page_views_page_views_last_week_log,resource_page_views_page_views_total,resource_page_views_page_views_total_log,resource_parent_fxf,resource_provenance,resource_publication_date,resource_type,resource_updatedAt
0,"[economy, environment, housing & development]",Housing & Development,"[{'key': 'Update_Automation', 'value': 'Yes'}, {'key': 'Update_Date-Made-Public', 'value': '4/26/2013'}, {'key': 'Update_Update-Frequency', 'value': 'Daily'}, {'key': 'Dataset-Information_Agency', 'value': 'Department of Buildings (DOB)'}]",NaN,"[job, dob, buildings]",[],https://data.cityofnewyork.us/Housing-Development/DOB-Job-Application-Filings/ic3t-wcy2,NaN,data.cityofnewyork.us,NaN,NYC OpenData,5fuc-pqz2,https://data.cityofnewyork.us/d/ic3t-wcy2,NaN,Department of Buildings (DOB),None,None,"[text, text, text, text, text, text, text, text, text, text, text, calendar_date, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text, text...","[Number assigned by DOB to Job Filing, Document Number, 1= Manhattan, 2= Bronx, 3 = Brooklyn, 4 = Queens, 5 = Staten Island, House Number of Residence or Commercial Property, Street Name where Property is located, Tax block assigned by Department of Finance, Tax lot assigned by Department of Fin...","[job__, doc__, borough, house__, street_name, block, lot, bin__, job_type, job_status, job_status_descrp, latest_action_date, building_type, community___board, cluster, landmarked, adult_estab, loft_board, city_owned, little_e, pc_filed, efiling_filed, plumbing, mechanical, boiler, fuel_burning,...","[{'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'right'}, {'align': 'righ...","[Job #, Doc #, Borough, House #, Street Name, Block, Lot, Bin #, Job Type, Job Status, Job Status Descrp, Latest Action Date, Building Type, Community - Board, Cluster, Landmarked, Adult Estab, Loft Board, City Owned, Little e, PC Filed, eFiling Filed, Plumbing, Mechanical, Boiler, Fuel Burning,...",None,2013-04-18T15:18:56.000Z,2020-02-20T21:07:37.000Z,"This dataset contains all job applications submitted through the Borough Offices, through eFiling, or through the HUB, which have a ""Latest Action Date"" since January 1, 2000. This dataset does not include jobs submitted through DOB NOW. See the DOB NOW: Build – Job Application Filings dataset f...",29689,False,ic3t-wcy2,tabular,2019-06-20T12:35:37.000Z,DOB Job Application Filings,3732,11.866120,989,9.951285,2233742,21.091032,None,official,2018-05-25T21:36:10.000Z,dataset,2020-02-20T21:07:37.000Z
1,"[transportation, environment]",Transportation,"[{'key': 'Update_Automation', 'value': 'Yes'}, {'key': 'Update_Date-Made-Public', 'value': '6/24/2016'}, {'key': 'Update_Update-Frequency', 'value': 'Several times per day'}, {'key': 'Legislative-Compliance_Geo-coding-Attached?', 'val

In [ ]:
json_normalize(tracks_response['tracks'], sep="_")


json_normalize(tracks_response['tracks'],record_path=['artists'],sep="_")

artist_and_track = json_normalize(
    data=tracks_response['tracks'],
    record_path='artists',
    meta=['id'],
    record_prefix='sp_artist_',
    meta_prefix='sp_track_',
    sep="_"
)
artist_and_track = artist_and_track[['sp_track_id','sp_artist_id']]